In [1]:
#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=9 #lattice dim
m=n+1 #space dim

R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]


#if n=9 generating forbidden nodes is time consuming
#so it is quicker to load the supplied file, may need to be unzipped first

import pickle
if n==9:
    with open('permutahedron lattice sublattice n9.p','rb') as f:
        (short,forb,forbd,forbdv) = pickle.load(f)
else:
    #first we generate all nodes with norm <=4R, and with norm<=2R
    short = []
    vshort = []
    K = ceil(4*R)
    K2 = (4*R)^2
    Kv = (2*R)^2

    indexes = [] #arrays of integers between -K and K having the same residue modulo m 
    for rem in range(m):
        res = []
        for i in range(-K,K+1):
            if i%m==rem:
                res.append(i)
        indexes.append(res)

    for c in range(0,K+1):
        for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
            p = [c]+list(coefs)+[-c-sum(coefs)]
            is_sorted = True
            for i in range(m-1):
                if p[i]<p[i+1]:
                    is_sorted = False
                    break
            if is_sorted:
                norm2 = sum((p[i])^2 for i in range(m))
                if norm2>0:
                    if norm2 <= Kv:
                        vshort.append(p)
                    if norm2 <= K2:
                        short.append(p)           


    #defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
    ineqs = [] 
    for i in range(m-1):
        toadd = [0]*(m+1)
        toadd[i+1]=1
        toadd[i+2]=-1
        ineqs.append(toadd) 

    ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

    #now Voronoi cell planes
    for p in vshort: #suffices to look at nodes at most 2R
        ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

    V = Polyhedron(ieqs=ineqs)

    ### orthogonal projection affine operator onto (improper) non-empty polytope 
    zerom = matrix(m)
    def projp(P):
        nv = P.n_vertices()
        z = vector(P.vertices()[0])
        if nv==1:
            return zerom,z
        A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
        return A*A.pseudoinverse(),z


    forb = []

    for d in range(0,n): #dim of face
        for ff in V.faces(d):
            proj = projp(ff)
            for p in short:
                if p not in forb:
                    x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                    proj_pnt = proj[0]*(x-proj[1])+proj[1]
                    if proj_pnt in ff.as_polyhedron():
                        dist = (x-proj_pnt).norm()
                        if dist<R:
                            forb.append(p)



    ### forbd = forb with possible permutations
    forbd = {}
    forbdv = []
    for x in forb:
        for t in Permutations(list(x)):
            c = vector(basis_coefs(t))
            if str(c) not in forbd:
                forbd[str(c)]=0
                if sum(c)>=0:
                    forbdv.append(c)
    ###

print("short:",len(short))
print("forbidden:",len(forb))
print("forbidden without symmetries:",len(forbd))

# saving
# with open('permutahedron lattice sublattice n9.p','wb') as f:
#     pickle.dump((short,forb,forbd,forbdv),f)


def check_sublattice(basis,target=Infinity): 
    A = matrix(basis).transpose() 

    Ad = abs(A.det())
    if Ad==0 or Ad>=target:
        return Infinity

    Ai = A.inverse()
    for x in forbdv:
        y = Ai*x
        allint = True
        for t in y:
            if t!=floor(t):
                allint = False
                break
        if allint:
            return Infinity   
    
    return Ad

if n==5:
    print(check_sublattice([[-2, -3, 0, -2, -2],
                            [1, 1, 1, 0, -2],
                            [-2, 0, 1, -2, 0],
                            [-1, -1, -1, 2, 0],
                            [0, -2, -3, -2, -2]]))   
    

if n==9:
    print(check_sublattice([[ 0, 1, 0, 0, 0, 3, 0, 0, -1 ],
    [ 0, 0, 0, 0, 3, 0, 0, 0, 0 ],
    [ -3, -3, -2, -3, -3, -3, -4, -3, -3 ],
    [ 1, 1, 1, 4, 1, 1, 2, 1, 1 ],
    [ 0, 1, 0, 0, 0, 0, 0, 3, -1 ],
    [ 0, 0, -1, 0, 0, 0, 3, 0, 1 ],
    [ -1, -1, -1, -1, -1, 2, -1, -1, 1 ],
    [ 1, 4, 1, 1, 1, 1, 2, 1, 1 ],
    [ 0, 1, 3, 0, 0, 0, 0, 0, -1 ]]))



short: 219
forbidden: 151
forbidden without symmetries: 778774
17253
